# Chipotle
> This notebook downloads the company's [news releases](https://ir.chipotle.com/news-releases?o=0) over time. 

#### Load Python tools and Jupyter config

In [28]:
import json
import requests
import pandas as pd
import jupyter_black
import altair as alt
import geopandas as gpd
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

In [2]:
jupyter_black.load()
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

In [3]:
today = pd.Timestamp("today").strftime("%Y%m%d")

---

## Fetch

#### Headers

In [4]:
headers = {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
}

#### Loop through the news release pages, snag metadata about them

In [29]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

headers = {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
}

# Initialize an empty list to store the press release metadata
data = []

# Loop over the paginated pages
for r in tqdm(range(0, 361, 5)):
    url = f"https://ir.chipotle.com/news-releases?o={r}"
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        html_content = BeautifulSoup(response.text, "html.parser")
        releases = html_content.find_all("div", class_="wd_item_wrapper")

        for release in releases:
            date = release.find("div", class_="wd_date").text.strip()
            title_tag = release.find("div", class_="wd_title").find("a")
            title = title_tag.text.strip()
            link = title_tag["href"].strip()

            # Append the data to the list
            data.append({"date": date, "title": title, "url": link})
    else:
        print(f"Failed to retrieve data from page {r}")

meta_df = pd.DataFrame(data)
meta_df["title"] = meta_df["title"].str.title()

# Filter for annual results URLs
annual_results_urls = meta_df.query(
    'title.str.contains("Announces") and title.str.contains("Full Year")'
)["url"].to_list()

  0%|          | 0/73 [00:00<?, ?it/s]

#### Convert the list to a DataFrame

In [113]:
# Initialize an empty list to store the table data
table_data = []

# Loop over the filtered URLs for annual results
for url in tqdm(annual_results_urls):
    press_release_response = requests.get(url, headers=headers)
    press_release_content = BeautifulSoup(press_release_response.text, "html.parser")
    tables = press_release_content.find_all("table", class_="prnbcc")

    if len(tables) > 1:  # Ensure there are at least two tables
        annual_income_statement_table = tables[1]
        rows = annual_income_statement_table.find_all("tr")

        for row in rows:
            cols = row.find_all("td")
            cols = [ele.text.strip() for ele in cols]
            table_data.append([url] + cols)

# Define column names based on the structure of the table
# column_names = [
#     "url",
#     "metric", "value_1", "percentage_1", "value_2", "percentage_2"
# ]

# Convert the list to a DataFrame
df = pd.DataFrame(table_data)

  0%|          | 0/7 [00:00<?, ?it/s]

In [114]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,https://ir.chipotle.com/2024-02-06-CHIPOTLE-ANNOUNCES-FOURTH-QUARTER-AND-FULL-YEAR-2023-RESULTS,"CHIPOTLE MEXICAN GRILL, INC.\nCONSOLIDATED STATEMENTS OF INCOME\n(in thousands, except per share data)\n(unaudited)",None,None,None,None,None,None,None,None,None,None,None
1,https://ir.chipotle.com/2024-02-06-CHIPOTLE-ANNOUNCES-FOURTH-QUARTER-AND-FULL-YEAR-2023-RESULTS,,,,,,,,,,,,
2,https://ir.chipotle.com/2024-02-06-CHIPOTLE-ANNOUNCES-FOURTH-QUARTER-AND-FULL-YEAR-2023-RESULTS,,"Three months ended December 31,",None,None,None,None,None,None,None,None,None,None
3,https://ir.chipotle.com/2024-02-06-CHIPOTLE-ANNOUNCES-FOURTH-QUARTER-AND-FULL-YEAR-2023-RESULTS,,2023,,2022,None,None,None,None,None,None,None,None
4,https://ir.chipotle.com/2024-02-06-CHIPOTLE-ANNOUNCES-FOURTH-QUARTER-AND-FULL-YEAR-2023-RESULTS,Food and beverage revenue,$,"2,499,567",,99.3,%,,$,"2,163,907",,99.2,%
5,https://ir.chipotle.com/2024-02-06-CHIPOTLE-ANNOUNCES-FOURTH-QUARTER-AND-FULL-YEAR-2023-RESULTS,Delivery service revenue,,"16,753",,0.7,,,,"16,692",,0.8,
6,https://ir.chipotle.com/2024-02-06-CHIPOTLE-ANNOUNCES-FOURTH-QUARTER-AND-FULL-YEAR-2023-RESULTS,Total revenue,,"2,516,320",,100.0,,,,"2,180,599",,100.0,
7,https://ir.chipotle.com/2024-02-06-CHIPOTLE-ANNOUNCES-FOURTH-QUARTER-AND-FULL-YEAR-2023-RESULTS,Restaurant operating costs (exclusive of depreciation and amortization shown separately below):,,,,,,,,,,,
8,https://ir.chipotle.com/2024-02-06-CHIPOTLE-ANNOUNCES-FOURTH-QUARTER-AND-FULL-YEAR-2023-RESULTS,"Food, beverage and packaging",,"747,155",,29.7,,,,"638,851",,29.3,
9,https://ir.chipotle.com/2024-02-06-CHIPOTLE-ANNOUNCES-FOURTH-QUARTER-AND-FULL-YEAR-2023-RESULTS,Labor,,"629,228",,25.0,,,,"558,914",,25.6,


['https://ir.chipotle.com/2024-02-06-CHIPOTLE-ANNOUNCES-FOURTH-QUARTER-AND-FULL-YEAR-2023-RESULTS',
 'https://ir.chipotle.com/2023-02-07-CHIPOTLE-ANNOUNCES-FOURTH-QUARTER-AND-FULL-YEAR-2022-RESULTS',
 'https://ir.chipotle.com/2022-02-08-CHIPOTLE-ANNOUNCES-FOURTH-QUARTER-AND-FULL-YEAR-2021-RESULTS',
 'https://ir.chipotle.com/2021-02-02-Chipotle-Announces-Fourth-Quarter-And-Full-Year-2020-Results',
 'https://ir.chipotle.com/2020-02-04-Chipotle-Announces-Fourth-Quarter-And-Full-Year-2019-Results',
 'https://ir.chipotle.com/news-releases?item=122415',
 'https://ir.chipotle.com/news-releases?item=122452']

In [110]:
headers = {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
}

# Initialize an empty list to store the table data
table_data = []

# Loop over the paginated pages
for r in tqdm(range(0, 361, 5)):  # Adjusting step to 5 assuming each page has 5 press releases
    url = f"https://ir.chipotle.com/news-releases?o={r}"
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        html_content = BeautifulSoup(response.text, "html.parser")
        releases = html_content.find_all("div", class_="wd_item_wrapper")
        
        for release in releases:
            date = release.find("div", class_="wd_date").text.strip()
            title_tag = release.find("div", class_="wd_title").find("a")
            title = title_tag.text.strip()
            link = title_tag["href"].strip()
            
            # Fetch the detailed press release page to extract the table
            press_release_response = requests.get(link, headers=headers)
            press_release_content = BeautifulSoup(press_release_response.text, "html.parser")
            tables = press_release_content.find_all("table", class_="prnbcc")
            
            if len(tables) > 1:  # Ensure there are at least two tables
                annual_income_statement_table = tables[1]
                rows = annual_income_statement_table.find_all("tr")
                
                for row in rows:
                    cols = row.find_all("td")
                    cols = [ele.text.strip() for ele in cols]
                    table_data.append([date, title, link] + cols)
    else:
        print(f"Failed to retrieve data from page {r}")

# Define column names based on the structure of the table
column_names = [
    "date", "title", "url",
    "metric", "value_1", "percentage_1", "value_2", "percentage_2"
]

# Convert the list to a DataFrame
df = pd.DataFrame(table_data, columns=column_names)

# Display the DataFrame
print(df)

# Optionally, save the DataFrame to a CSV file
df.to_csv("chipotle_press_release_annual_income_statements.csv", index=False)


  0%|          | 0/73 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [74]:
df = pd.concat(dfs)

In [77]:
df.dropna(thresh=8)

,0,1,2,3,4,5,6,7,8,9,10,11,url,12
0,"CHIPOTLE MEXICAN GRILL, INC. CONSOLIDATED STATEMENTS OF INCOME (in thousands, except per share data)","CHIPOTLE MEXICAN GRILL, INC. CONSOLIDATED STATEMENTS OF INCOME (in thousands, except per share data)","CHIPOTLE MEXICAN GRILL, INC. CONSOLIDATED STATEMENTS OF INCOME (in thousands, except per share data)","CHIPOTLE MEXICAN GRILL, INC. CONSOLIDATED STATEMENTS OF INCOME (in thousands, except per share data)","CHIPOTLE MEXICAN GRILL, INC. CONSOLIDATED STATEMENTS OF INCOME (in thousands, except per share data)","CHIPOTLE MEXICAN GRILL, INC. CONSOLIDATED STATEMENTS OF INCOME (in thousands, except per share data)","CHIPOTLE MEXICAN GRILL, INC. CONSOLIDATED STATEMENTS OF INCOME (in thousands, except per share data)","CHIPOTLE MEXICAN GRILL, INC. CONSOLIDATED STATEMENTS OF INCOME (in thousands, except per share data)","CHIPOTLE MEXICAN GRILL, INC. CONSOLIDATED STATEMENTS OF INCOME (in thousands, except per share data)","CHIPOTLE MEXICAN GRILL, INC. CONSOLIDATED STATEMENTS OF INCOME (in thousands, except per share data)","CHIPOTLE MEXICAN GRILL, INC. CONSOLIDATED STATEMENTS OF INCOME (in thousands, except per share data)","CHIPOTLE MEXICAN GRILL, INC. CONSOLIDATED STATEMENTS OF INCOME (in thousands, except per share data)",https://ir.chipotle.com/2024-02-06-CHIPOTLE-ANNOUNCES-FOURTH-QUARTER-AND-FULL-YEAR-2023-RESULTS,NaN
2,NaN,"Year ended December 31,","Year ended December 31,","Year ended December 31,","Year ended December 31,","Year ended December 31,","Year ended December 31,","Year ended December 31,","Year ended December 31,","Year ended December 31,","Year ended December 31,","Year ended December 31,",https://ir.chipotle.com/2024-02-06-CHIPOTLE-ANNOUNCES-FOURTH-QUARTER-AND-FULL-YEAR-2023-RESULTS,NaN
3,NaN,2023,2023,2023,2023,2023,NaN,2022,2022,2022,2022,2022,https://ir.chipotle.com/2024-02-06-CHIPOTLE-ANNOUNCES-FOURTH-QUARTER-AND-FULL-YEAR-2023-RESULTS,NaN
5,Food and beverage revenue,$,9804124,NaN,99.3,%,NaN,$,8558001,NaN,99.1,%,https://ir.chipotle.com/2024-02-06-CHIPOTLE-ANNOUNCES-FOURTH-QUARTER-AND-FULL-YEAR-2023-RESULTS,NaN
22,Net income,$,1228737,NaN,12.4,%,NaN,$,899101,NaN,10.4,%,https://ir.chipotle.com/2024-02-06-CHIPOTLE-ANNOUNCES-FOURTH-QUARTER-AND-FULL-YEAR-2023-RESULTS,NaN
0,"CHIPOTLE MEXICAN GRILL, INC. CONSOLIDATED STATEMENTS OF INCOME (in thousands, except per share data)","CHIPOTLE MEXICAN GRILL, INC. CONSOLIDATED STATEMENTS OF INCOME (in thousands, except per share data)","CHIPOTLE MEXICAN GRILL, INC. CONSOLIDATED STATEMENTS OF INCOME (in thousands, except per share data)","CHIPOTLE MEXICAN GRILL, INC. CONSOLIDATED STATEMENTS OF INCOME (in thousands, except per share data)","CHIPOTLE MEXICAN GRILL, INC. CONSOLIDATED STATEMENTS OF INCOME (in thousands, except per share data)","CHIPOTLE MEXICAN GRILL, INC. CONSOLIDATED STATEMENTS OF INCOME (in thousands, except per share data)","CHIPOTLE MEXICAN GRILL, INC. CONSOLIDATED STATEMENTS OF INCOME (in thousands, except per share data)","CHIPOTLE MEXICAN GRILL, INC. CONSOLIDATED STATEMENTS OF INCOME (in thousands, except per share data)","CHIPOTLE MEXICAN GRILL, INC. CONSOLIDATED STATEMENTS OF INCOME (in thousands, except per share data)","CHIPOTLE MEXICAN GRILL, INC. CONSOLIDATED STATEMENTS OF INCOME (in thousands, except per share data)","CHIPOTLE MEXICAN GRILL, INC. CONSOLIDATED STATEMENTS OF INCOME (in thousands, except per share data)","CHIPOTLE MEXICAN GRILL, INC. CONSOLIDATED STATEMENTS OF INCOME (in thousands, except per share data)",https://ir.chipotle.com/2023-02-07-CHIPOTLE-ANNOUNCES-FOURTH-QUARTER-AND-FULL-YEAR-2022-RESULTS,NaN
2,NaN,"Year ended December 31,","Year ended December 31,","Year ended December 31,","Year ended December 31,","Year ended December 31,","Year ended December 31,","Year ended December 31,","Year ended December 31,","Year ended December 31,","Year ended December 31,","Year ended December 31,",https://ir.chipotle.com/2023-02-07-CHIPOTLE-ANNOUNCES-FOURTH-QUARTER-AND-FULL-YEAR-2022-RESULTS,NaN
3,NaN,2022,20

---

## Exports

#### JSON

In [17]:
# df.to_json(
#     f"data/processed/NAME.json",
#     indent=4,
#     orient="records",
# )

#### CSV

In [18]:
# Optionally, save the DataFrame to a CSV file
df.to_csv("data/processed/chipotle_press_releases.csv", index=False)

#### GeoJSON

In [19]:
# gdf.to_file(
#     f"data/processed/NAME.geojson",
#     driver="GeoJSON",
# )